# Documentation:
## Objective:
Find variables that helps to explain product Salesrank in Amazon

## What things this notebook has done?
Change tabular data into item-wise dictionary

Metrics generation:

Calculation: aggregation metrics (Average ratings)
Date: time metrics
Text: average word count
Regression basedline model

## What's the dataset about?
Earthlite's product reviews data from Amazon
Sales Rank of Earthlite products under different categories

## Run time

## Common bugs:

## Below are detailed code chunks

# Part 1: Setup working environment & Dataset
Load sales rank & review dataset for Earthlite product

In [1]:
debug=True
option=2

In [2]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

#!pip install quilt
#!quilt install ResidentMario/missingno_data
from quilt.data.ResidentMario import missingno_data

%matplotlib inline

In [3]:
# Earthlite review data
hmr = pd.read_csv('Earthlite Reviews.csv')
# Earthlite sales rank data
hmsr = pd.read_csv('Earthlite Sales Rank.csv')
# Earthlite questions data
hmq = pd.read_csv('Earthlite Questions.csv')
# Earthlite price data
hmp = pd.read_csv('Earthlite Price.csv')

/Users/yvettepeng/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
hmr.columns

Index(['source', 'product', 'Brand_custom', 'COLOR_custom', 'Category_custom',
       'Country_custom', 'Name_custom', 'UPC_custom', 'model_number_custom',
       'name', 'date', 'status', 'sentiment', 'topic', 'notes', 'author',
       'verified', 'vine', 'stars', 'pvotes', 'tvotes', 'title', 'text',
       'image', 'video', 'reviewid', 'reviewlink', 'parent', 'statusdt',
       'statustime', 'helpfulcount', 'commenttext', 'commentauthor',
       'officialcomment', 'totalcomments', 'commentts', 'commentdatestring',
       'inputtime'],
      dtype='object')

In [5]:
hmsr.columns

Index(['source', 'id', 'COLOR_custom', 'Name_custom', 'model_number_custom',
       'start_ts', 'end_ts', 'date', 'category_id1', 'category_name1',
       'category_rank1', 'category_id2', 'category_name2', 'category_rank2',
       'category_id3', 'category_name3', 'category_rank3'],
      dtype='object')

In [6]:
# select one product category that has most products
hmsr.groupby(by='category_id1', as_index=False).agg({'id': pd.Series.nunique}).sort_values(by = 'id', ascending=False)

,category_id1,id
7,beauty/7792506011,177
6,beauty/16302991,88
3,beauty/15144892011,17
2,beauty/15144890011,13
4,beauty/16302961,8
1,beauty/15144566011,6
8,home-garden/16351481,3
10,lawn-garden/1272941011,3
0,beauty/15003720011,2
5,beauty/16302971,2


In [8]:
# Select 10 option 1 products
hmsr[hmsr['category_id1'] == 'beauty/7792506011'][['id', 'category_rank1']].drop_duplicates().groupby('id').std().sort_values(by = 'category_rank1', ascending = False).head(10)

,category_rank1
id,
B0016JL1C0,503.289346
B0016JJH4E,449.689458
B0016JKZAE,436.065940
B0016JN0PQ,379.597161
B0016JJIAM,378.196900
B0016JN0MO,354.134819
B0016JIEJS,298.847093
B0016JJIS4,285.971439
B0016JJIZ2,282.530437


In [9]:
## select 10 option 2 products
hmr[hmr['product'].isin(hmsr[hmsr['category_id1'] == 'beauty/7792506011']['id'].unique())][['product','reviewid']].groupby('product').nunique().sort_values(by = 'reviewid', ascending = False).head(10)

,product,reviewid
product,,
B004GG9ECY,1,190
B00122GAJK,1,180
B00122NEIA,1,109
B00122I6UQ,1,74
B004GG62TM,1,68
B00122JEFC,1,68
B00BG84X26,1,63
B004GG9R8K,1,45
B00122JEF2,1,42


# Options for modeling:
Group 1: more variability
Group 2: most reviews records
Must under missing_value check

In [10]:
if(option==1):
    # more reviews and highly variant sales rank
    common = ['B0016JL1C0',
             'B0016JJH4E',
             'B0016JKZAE',
             'B0016JN0PQ',
             'B0016JJIAM',
             'B0016JN0MO',
             'B0016JIEJS',
             'B0016JJIS4',
             'B0016JJIZ2',
            'B0016JKZIG']
elif(option==2):
    # products with most reviews
    common=['B004GG9ECY', 
         'B00122GAJK', 
         'B00122NEIA', 
         'B00122I6UQ',
         'B004GG62TM',
         'B00122JEFC',
           'B00BG84X26',
           'B004GG9R8K',
           'B00122JEF2',
           'B00122JD6W']
else:
    pass

In [11]:
if(debug):
    hmsr2 = hmsr[hmsr['id'].isin(common)].set_index('id',drop=True)
    #hmsr = hmsr.set_index('id',drop = True)
    hmr2 = hmr[hmr['product'].isin(common)].set_index('product',drop=True)
    hmp2 = hmp[hmp['id'].isin(common)].set_index('id',drop=True)
else:
    hmsr2 = hmsr.set_index('id',drop = True)
    # Same as above, create another dictionary for review data
    hmr2 = hmr.set_index('product',drop = True)
    hmp2 = hmp.set_index('id',drop=True)

### Transform into key-value pairs by each unique product ID
To make aggregation on review ratings, we need details on each product

In [12]:
# create a dictionary to store the information for each product. Each product represents one element of the dictionary.
hmsr_dict = {}
for i in hmsr2.index.unique():
    hmsr_dict[i] = hmsr2.loc[i, ['source','date', 'category_id1',
       'category_name1', 'category_rank1', 'category_id2', 'category_name2',
       'category_rank2', 'category_id3', 'category_name3', 'category_rank3',
       'category_id4', 'category_name4', 'category_rank4']]

/Users/yvettepeng/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [13]:
item_list=list(hmsr_dict.keys())
item_list[1]

'B00122GAJK'

### Features Time based rank Features
* log_rank 
* delta_rank
* date

#### Sales rank dataset

In [14]:
for i in hmsr_dict:
    if type(hmsr_dict[i]) != pd.core.series.Series:
        hmsr_dict[i] = hmsr_dict[i].sort_values('date').reset_index()
        hmsr_dict[i].drop_duplicates(inplace = True)
        hmsr_dict[i]['new_date'] = pd.to_datetime(hmsr_dict[i]['date'])
        hmsr_dict[i]['log_rank'] = hmsr_dict[i]['category_rank1'].map(lambda x: math.log(x))
        hmsr_dict[i]['delta_rank'] = hmsr_dict[i]['log_rank'].diff()
        a = hmsr_dict[i].index[0]
        hmsr_dict[i].drop(a, inplace = True)
        hmsr_dict[i]['new_date'] = hmsr_dict[i]['new_date'].dt.strftime('%Y-%m-%d')
        hmsr_dict[i]['new_date'] = pd.to_datetime(hmsr_dict[i]['new_date'])

In [15]:
for i in hmsr_dict:
    if type(hmsr_dict[i]) != pd.core.series.Series:
        hmsr_dict[i] = hmsr_dict[i].reset_index(drop = True)

#### Review dataset

In [16]:
hmr_dict = {}
for i in hmr2.index.unique():
    hmr_dict[i] = hmr2.loc[i, ['source','date','author', 'verified', 'vine', 'stars', 'pvotes', 'tvotes',
       'title', 'text', 'image', 'video']]

In [17]:
for i in hmr_dict:
    if type(hmr_dict[i]) != pd.core.series.Series:
        hmr_dict[i] = hmr_dict[i].sort_values('date').reset_index()
        hmr_dict[i].drop_duplicates(inplace = True)
        hmr_dict[i]['date'] = pd.to_datetime(hmr_dict[i]['date'])
        hmr_dict[i]['date'] = hmr_dict[i]['date'].dt.strftime('%Y-%m-%d')
        hmr_dict[i]['date'] = pd.to_datetime(hmr_dict[i]['date'])

In [18]:
for i in hmr_dict:
    if type(hmr_dict[i]) != pd.core.series.Series:
        hmr_dict[i] = hmr_dict[i].reset_index(drop = True)

#### Price dataset

In [19]:
hmp_dict = {}
for i in hmp2.index.unique():
    hmp_dict[i] = hmp2.loc[i, ['id','buyboxtime','price',
       'high', 'low', 'stock', 'merchant','prime','subscribe',
       'pantry', 'addon', 'min', 'max', 'mapviolation', 'sellers',
       'Category_custom', 'Subcategory_custom']]

In [20]:
for i in hmp_dict:
    if type(hmp_dict[i]) != pd.core.series.Series:
        hmp_dict[i] = hmp_dict[i].drop(labels = 'id', axis = 1)
        hmp_dict[i] = hmp_dict[i].sort_values('buyboxtime').reset_index()
        hmp_dict[i].drop_duplicates(inplace = True)
        hmp_dict[i]['date'] = pd.to_datetime(hmp_dict[i]['buyboxtime'])
        hmp_dict[i]['date'] = hmp_dict[i]['date'].dt.strftime('%Y-%m-%d')
        hmp_dict[i]['date'] = pd.to_datetime(hmp_dict[i]['date'])

### 2.1 Feature group: aggregated features

In [22]:
# Now that we have created two dictionaries, next we are going to do the aggregation
for i in hmsr_dict:
    if i in hmr_dict:
        for j in hmsr_dict[i].index.unique():
            if type(hmr_dict[i]) != pd.core.series.Series:
                hmsr_dict[i].loc[j, "stars_recent_10"] = hmr_dict[i][hmr_dict[i]['date'] <= hmsr_dict[i].loc[j, 'date']].tail(10)['stars'].mean()

In [23]:
for i in hmsr_dict:
    if i in hmr_dict:
        for j in hmsr_dict[i].index.unique():
            if type(hmr_dict[i]) != pd.core.series.Series:
                hmsr_dict[i].loc[j, 'stars_recent_oneweek'] = hmr_dict[i][(hmr_dict[i]['date'] <= 
                                                                           hmsr_dict[i].loc[j, 'new_date']) & (hmr_dict[i]['date'] >= (hmsr_dict[i].loc[j,'new_date'] - datetime.timedelta(days = 6)))]['stars'].mean()
                
                
                hmsr_dict[i].loc[j, 'stars_recent_onemonth'] = hmr_dict[i][(hmr_dict[i]['date'] <= 
                                                                          hmsr_dict[i].loc[j, 'new_date']) & (hmr_dict[i]['date'] >= (hmsr_dict[i].loc[j,'new_date'] - datetime.timedelta(days = 29)))]['stars'].mean()
                
                
                hmsr_dict[i].loc[j, 'stars_avg'] = hmr_dict[i][hmr_dict[i]['date'] <= hmsr_dict[i].loc[j, 'new_date']]['stars'].mean()
                
                hmsr_dict[i].loc[j, 'review_acc'] = hmr_dict[i][hmr_dict[i]['date'] <= hmsr_dict[i].loc[j, 'new_date']]['date'].count()
                
                hmsr_dict[i].loc[j, 'reviewnum_oneweek'] = hmr_dict[i][(hmr_dict[i]['date'] <= 
                                                                          hmsr_dict[i].loc[j, 'new_date']) & (hmr_dict[i]['date'] >= (hmsr_dict[i].loc[j,'new_date'] - datetime.timedelta(days = 6)))]['date'].count()
                
                
                hmsr_dict[i].loc[j, 'reviewnum_onemonth'] = hmr_dict[i][(hmr_dict[i]['date'] <= 
                                                                          hmsr_dict[i].loc[j, 'new_date']) & (hmr_dict[i]['date'] >= (hmsr_dict[i].loc[j,'new_date'] - datetime.timedelta(days = 29)))]['date'].count()
                hmsr_dict[i].loc[j, 'author_acc'] = hmr_dict[i][(hmr_dict[i]['date'] <= hmsr_dict[i].loc[j,'new_date']) & 
                                                (hmr_dict[i]['author'].str.contains('Customer') == False) & (hmr_dict[i]['author'].str.contains('customer') == False)]['date'].count() /  hmr_dict[i][hmr_dict[i]['date'] <= hmsr_dict[i].loc[j,'new_date']]['date'].count()
                hmsr_dict[i].loc[j, 'num_1_month'] = hmr_dict[i][(hmr_dict[i]['date'] <= 
                                                                          hmsr_dict[i].loc[j, 'date']) & (hmr_dict[i]['date'] >= (hmsr_dict[i].loc[j,'new_date'] - datetime.timedelta(days = 29))) & (hmr_dict[i]['stars'] == 1)]['date'].count()
                hmsr_dict[i].loc[j, 'num_5_month'] = hmr_dict[i][(hmr_dict[i]['date'] <= 
                                                                          hmsr_dict[i].loc[j, 'date']) & (hmr_dict[i]['date'] >= (hmsr_dict[i].loc[j,'new_date'] - datetime.timedelta(days = 29))) & (hmr_dict[i]['stars'] == 5)]['date'].count()
                

### 2.2 Text related Features:
* average word count

In [24]:
# of words in the reviews
# Generate a new column: average word count of review that happen on the same sales rank day
# for i in df_review2.index.unique():
#     df_review2.loc[i,'word_count'] = df_review2.loc[i,'text'].count(' ') + 1
  
for i in hmsr_dict:
    try:
        if i in hmr_dict:
            for j in hmsr_dict[i].index.unique():
                if type(hmr_dict[i]) != pd.core.series.Series:
                    hmsr_dict[i].loc[j, 'avg_word_count'] = hmr_dict[i][hmr_dict[i]['date'] <= hmsr_dict[i].loc[j, 'date']]['text'].apply(lambda x: x.count(' ')+1).mean()
                    hmsr_dict[i].loc[j, 'avg_word_count_10'] = hmr_dict[i][hmr_dict[i]['date'] <= hmsr_dict[i].loc[j, 'date']].tail(10)['text'].apply(lambda x: x.count(' ')+1).mean()        
    except:
        continue   

### 2.3 Features: reviewer status
* all: count of varified_account reviews 
* past one week: count of varified_account reviews

In [25]:
for i in hmsr_dict:
    if i in hmr_dict:
        for j in hmsr_dict[i].index.unique():
            if type(hmr_dict[i]) != pd.core.series.Series:
                hmsr_dict[i].loc[j, 'verified_acc'] = (hmr_dict[i][(hmr_dict[i]['date'] <= hmsr_dict[i].loc[j, 'date']) & (hmr_dict[i]['verified'] == True)]['date'].count()) / (hmr_dict[i][hmr_dict[i]['date'] <= hmsr_dict[i].loc[j, 'date']]['date'].count()) 
            if type(hmr_dict[i]) != pd.core.series.Series:
                hmsr_dict[i].loc[j, 'verified_acc_oneweek'] = (hmr_dict[i][(hmr_dict[i]['date'] <= 
                                                                          hmsr_dict[i].loc[j, 'new_date']) & (hmr_dict[i]['date'] >= (hmsr_dict[i].loc[j,'new_date'] - datetime.timedelta(days = 6))) & (hmr_dict[i]['verified'] == True)]['date'].count()) / (hmr_dict[i][(hmr_dict[i]['date'] <= 
                                                                          hmsr_dict[i].loc[j, 'new_date']) & (hmr_dict[i]['date'] >= (hmsr_dict[i].loc[j,'new_date'] - datetime.timedelta(days = 6)))]['date'].count()) 

/Users/yvettepeng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':


### 2.4 Price related features
* number of sellers
* price

In [26]:
for i in hmsr_dict:
    if i in hmp_dict:
        for j in range(4,len(hmsr_dict[i].index)):
            if type(hmp_dict[i]) != pd.core.series.Series:
                hmsr_dict[i].loc[j, 'latest_price'] = hmp_dict[i][hmp_dict[i]['buyboxtime'] <= hmsr_dict[i].loc[j, 'date']]['price'].iloc[-1]
                
                

In [27]:
for i in hmsr_dict:
    if i in hmp_dict:
        for j in range(4,len(hmsr_dict[i].index)):
            if type(hmp_dict[i]) != pd.core.series.Series:
                hmsr_dict[i].loc[j, 'first_price'] = hmp_dict[i][hmp_dict[i]['date'] == hmsr_dict[i].loc[j, 'new_date']]['price'].iloc[-1]
                
                

In [28]:
# fill NAs with the mean price
for i in hmsr_dict:
    hmsr_dict[i]['latest_price'] = hmsr_dict[i]['latest_price'].fillna(hmsr_dict[i]['latest_price'].mean())
    hmsr_dict[i]['first_price'] = hmsr_dict[i]['first_price'].fillna(hmsr_dict[i]['first_price'].mean())

In [34]:
hmsr_dict['B00122JD6W']

,id,source,date,category_id1,category_name1,category_rank1,category_id2,category_name2,category_rank2,category_id3,...,reviewnum_onemonth,author_acc,num_1_month,num_5_month,avg_word_count,avg_word_count_10,verified_acc,verified_acc_oneweek,latest_price,first_price
0,B00122JD6W,amazon,2019-08-09 18:10:50,beauty/7792506011,Spa Beds & Tables,60.0,sporting-goods,Sports & Outdoors,46644.0,NaN,...,0.0,0.975,0.0,0.0,43.225,54.3,1.0,NaN,408.618941,408.618941
1,B00122JD6W,amazon,2019-08-10 18:58:51,beauty/7792506011,Spa Beds & Tables,126.0,sporting-goods,Sports & Outdoors,84066.0,NaN,...,0.0,0.975,0.0,0.0,43.225,54.3,1.0,NaN,408.618941,408.618941
2,B00122JD6W,amazon,2019-08-11 17:29:32,beauty/7792506011,Spa Beds & Tables,81.0,sporting-goods,Sports & Outdoors,68393.0,NaN,...,0.0,0.975,0.0,0.0,43.225,54.3,1.0,NaN,408.618941,408.618941
3,B00122JD6W,amazon,2019-08-12 13:30:36,beauty/7792506011,Spa Beds & Tables,66.0,sporting-goods,Sports & Outdoors,58901.0,NaN,...,0.0,0.975,0.0,0.0,43.225,54.3,1.0,NaN,408.618941,408.618941
4,B00122JD6W,amazon,2019-08-13 18:38:03,beauty/7792506011,Spa Beds & Tables,51.0,sporting-goods,Sports & Outdoors,39672.0,NaN,...,0.0,0.975,0.0,0.0,43.225,54.3,1.0,NaN,422.860000,422.860000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,B00122JD6W,amazon,2019-11-01 18:08:49,beauty/7792506011,Spa Beds & Tables,64.0,sporting-goods,Sports & Outdoors,46062.0,NaN,...,0.0,0.975,0.0,0.0,43.225,54.3,1.0,NaN,425.000000,425.000000
85,B00122JD6W,amazon,2019-11-02 17:48:02,beauty/7792506011,Spa Beds & Tables,83.0,sporting-goods,Sports & Outdoors,59594.0,NaN,...,0.0,0.975,0.0,0.0,43.225,54.3,1.0,NaN,425.000000,425.000000
86,B00122JD6W,amazon,2019-11-03 17:54:17,beauty/7792506011,Spa Beds & Tables,45.0,sporting-goods,Sports & Outdoors,33208.0,NaN,...,0.0,0.975,0.0,0.0,43.225,54.3,1.0,NaN,425.000000,425.000000
87,B00122JD6W,amazon,2019-11-04 18:38:12,beauty/7792506011,Spa Beds & Tables,80.0,sporting-goods,Sports & Outdoors,56654.0,NaN,...,0.0,0.975,0.0,0.0,43.225,54.3,1.0,NaN,425.000000,425.000000


#### Remove problematic products with only 1 records

In [35]:
problem_products=[]
for i in hmsr_dict:
    if type(hmsr_dict[i]['category_rank1'])!= pd.core.series.Series:
        problem_products.append(i)
problem_products

[]

In [36]:
for item in problem_products:
    del hmsr_dict[item]

### 2.5 Data preparation for modeling
Models for different product groups
    * Large salesrank variation
    * Most reviews records

In [37]:
# To change the keys from product ids to numbers, preparing for the following loop.
new_dict_hmsr = dict((i,hmsr_dict[k]) for i,k in enumerate(sorted(hmsr_dict.keys())))

# pull out interested products to form a new dataframe

a = new_dict_hmsr[0]
for i in new_dict_hmsr:
    new_dict_hmsr[i]=new_dict_hmsr[i].reset_index()
    if new_dict_hmsr[i].loc[1,'id'] in common: 
        a = pd.concat([a,new_dict_hmsr[i]],ignore_index=True)

/Users/yvettepeng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


In [38]:
a['id']

0      B00122GAJK
1      B00122GAJK
2      B00122GAJK
3      B00122GAJK
4      B00122GAJK
          ...    
962    B00BG84X26
963    B00BG84X26
964    B00BG84X26
965    B00BG84X26
966    B00BG84X26
Name: id, Length: 967, dtype: object

In [40]:
# The first product is what we don't want, (not in common), but in the above loop, it's more convenient to include that, so we delete it now.
# final_df = a[a['id'] != 'B00000JFNV']
final_df = a
# put our interested columns in the final dataframe.
final_df = final_df.reset_index(drop = True)[['id', 'date', 'category_name1',
       'category_rank1','log_rank', 'delta_rank',
       'stars_recent_10', 'stars_recent_oneweek',
       'stars_recent_onemonth', 'stars_avg', 'review_acc', 'reviewnum_oneweek',
       'reviewnum_onemonth', 'avg_word_count', 'verified_acc',
       'verified_acc_oneweek', 'log_rank', 'delta_rank','avg_word_count_10','num_1_month', 'num_5_month',
        'latest_price','first_price','author_acc']]


# replace the NAs with 0
final_df = final_df.fillna(0)
temp=final_df
final_df.drop_duplicates(inplace = True)
final_df.to_csv('final_df.csv')